In [1]:
import boto3
import json
from botocore.exceptions import ClientError

In [3]:
session = boto3.Session(profile_name='workshop')
config = session.client('config', region_name='us-west-2')

In [9]:
def create_custom_rule(rule_name, rule_description, resource_type, rule_content, input_parameters=None):
    rule = {
        'ConfigRuleName': rule_name,
        'Description': rule_description,
        'Scope': {
            'ComplianceResourceTypes': [
                resource_type,
            ],
        },
        'Source': {
            'Owner': 'CUSTOM_POLICY',
            'SourceDetails': [
                {
                    'EventSource': 'aws.config',
                    'MessageType': 'ConfigurationItemChangeNotification'
                },
            ],
            'CustomPolicyDetails': {
                'PolicyRuntime': 'guard-2.x.x',
                'PolicyText': rule_content,
                'EnableDebugLogDelivery': True,
            },
        }
    }
    if input_parameters is not None:
        rule['InputParameters'] = input_parameters
    try:
        response = config.put_config_rule(ConfigRule=rule)
        print('Successfully created Config rule: ' + json.dumps(response, indent=2, default=str))
    except ClientError as e:
        print('Error creating Config rule: ' + e.response['Error']['Code'])
    except Exception as e:
        print('Error creating Config rule: ' + str(e))

In [10]:
rule_name = 's3-bucket-versioning-enabled-test'
rule_description = 'Checks that S3 buckets have versioning enabled'
resource_type = 'AWS::S3::Bucket'
rule_content = """rule check_bucketversioning {
     supplementaryConfiguration.BucketVersioningConfiguration.status == "Enabled" <<
     result: NON_COMPLIANT
     message: S3 Bucket Versioning is NOT enabled.
     >>
}
"""
create_custom_rule(rule_name, rule_description, resource_type, rule_content)

Successfully created Config rule: {
  "ResponseMetadata": {
    "RequestId": "931a0ec1-7880-4281-addf-adca7746b822",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "931a0ec1-7880-4281-addf-adca7746b822",
      "strict-transport-security": "max-age=86400",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "0",
      "date": "Wed, 31 May 2023 02:16:21 GMT"
    },
    "RetryAttempts": 0
  }
}


In [5]:
rule_name = 'ec2-eip-running-test-new'
rule_description = 'Checks that EC2 EIPs are associated with running instances'
resource_type = 'AWS::EC2::Instance'
input_parameters = "{\"volumeType\":\"gp3\"}"
rule_content = """let eipresource = relationships.*[ resourceType  == 'AWS::EC2::EIP' ]


rule check_ec2_eip_compliance {
    when %eipresource !empty {
    configuration.state.name == "running"
}
}
"""
create_custom_rule(rule_name, rule_description, resource_type, rule_content, input_parameters)

Successfully created Config rule: {
  "ResponseMetadata": {
    "RequestId": "5138d6d7-4c41-44f7-bb3d-fd135dbbd800",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "5138d6d7-4c41-44f7-bb3d-fd135dbbd800",
      "strict-transport-security": "max-age=86400",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "0",
      "date": "Wed, 31 May 2023 02:09:17 GMT"
    },
    "RetryAttempts": 0
  }
}


In [11]:
# Upload a file to S3 bucket
s3_client = session.client('s3')
file = 'conf_pack.yaml'
response = s3_client.upload_file(file, 'config-bucket-949951905621', file)

In [13]:
# Create conformance pack
response = config.put_conformance_pack(
    ConformancePackName='conformance-pack-1',
    TemplateS3Uri='s3://config-bucket-949951905621/conf_pack.yaml',
    DeliveryS3Bucket='config-bucket-949951905621',
    TemplateBody='string',
    ConformancePackInputParameters=[
        {
            'ParameterName': 'SnsTopicForPublishNotificationArn',
            'ParameterValue': 'arn...'
        },
    ]
)
print(response)

{'ConformancePackArn': 'arn:aws:config:us-west-2:949951905621:conformance-pack/conformance-pack-1/conformance-pack-4dja86bgm', 'ResponseMetadata': {'RequestId': '8eeb7dfd-73ed-49cf-ad75-a7b758bdd529', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '8eeb7dfd-73ed-49cf-ad75-a7b758bdd529', 'strict-transport-security': 'max-age=86400', 'content-type': 'application/x-amz-json-1.1', 'content-length': '125', 'date': 'Wed, 31 May 2023 02:59:37 GMT'}, 'RetryAttempts': 0}}
